In [ ]:
# Importing necessary libraries
import networkx as nx
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import numpy as np
from scipy.optimize import minimize

In [ ]:
# Seat assignment functions (reuse code from previous file)

def seat_type_passenger(seattype):
    """
    Map seat type for passengers to corresponding labels ('A', 'B', 'C', 'D').
    """
    seat_mapping = {"BusinessClass": "A", "EconomyClass": "D", "PremiumEconomyClass": "C", "FirstClass": "B"}
    return seat_mapping.get(seattype, 'A')  # Default to 'A' if seat type is not recognized

def seat_type_flight(dataframe):
    """
    Rename seat types in flight dataframe to standard labels ('A', 'B', 'C', 'D').
    """
    dataframe.rename(columns={'BC_Oversold': 'A', 'EC_Oversold': 'D', 'PC_Oversold': 'C', 'FC_Oversold': 'B'}, inplace=True)
    return dataframe

def seat_type_flight_revert(dataframe):
    """
    Revert seat type labels in flight dataframe to original names.
    """
    dataframe.rename(columns={'A': 'BC_Oversold', 'D': 'EC_Oversold', 'C': 'PC_Oversold', 'B': 'FC_Oversold'}, inplace=True)
    return dataframe

def solve_weighted_bipartite(graph):
    """
    Solve the weighted bipartite matching problem using networkx library.
    """
    matching = nx.algorithms.max_weight_matching(graph, weight='weight', maxcardinality=True)
    return matching

def create_weighted_bipartite_graph(passengers, flights, upgrade_class=False, downgrade_class=True):
    """
    Create a weighted bipartite graph for the flight assignment problem.
    """
    G = nx.Graph()
    flights = seat_type_flight(flights)

    # Add nodes for passengers
    for _, passenger in passengers.iterrows():
        passenger['COS_CD'] = seat_type_passenger(passenger['COS_CD'])
        G.add_node(passenger['CUSTOMER_ID'], bipartite=0, seat_type=passenger['COS_CD'])

    # Add nodes for seats with the same seat type as flights
    for _, flight in flights.iterrows():
        for seat_type in ["A", "B", "C", "D"]:
            for seat_number in range(1, flight[seat_type] + 1):
                seat_name = f"{flight['FlightNumber']}_{seat_type}{seat_number}"
                G.add_node(seat_name, bipartite=1, seat_type=seat_type)

                # Add edges with weights equal to the sum of passenger and flight scores
                for _, passenger in passengers.iterrows():
                    passenger['COS_CD'] = seat_type_passenger(passenger['COS_CD'])
                    if passenger['COS_CD'] == seat_type:
                        edge_weight = passenger['Passenger_Rating'] + flight['Rating']
                        G.add_edge(passenger['CUSTOMER_ID'], seat_name, weight=edge_weight)

    flights = seat_type_flight_revert(flights)

    return G



# Sample data
passengers = pd.DataFrame([
    {'CUSTOMER_ID': 'Alice', 'COS_CD': 'BusinessClass', 'Passenger_Rating': 20},
    {'CUSTOMER_ID': 'Bob', 'COS_CD': 'EconomyClass', 'Passenger_Rating': 30},
    {'CUSTOMER_ID': 'Charlie', 'COS_CD': 'PremiumEconomyClass', 'Passenger_Rating': 25},
])

flights = pd.DataFrame([
    {'FlightNumber': 'F1', 'BC_Oversold': 5, 'EC_Oversold': 2, 'PC_Oversold': 3, 'FC_Oversold': 10, 'Rating': 10},
    {'FlightNumber': 'F2', 'BC_Oversold': 2, 'EC_Oversold': 3, 'PC_Oversold': 6, 'FC_Oversold': 5, 'Rating': 15},
])

# Create a weighted bipartite graph
weighted_bipartite_graph = create_weighted_bipartite_graph(passengers, flights, upgrade_class=True, downgrade_class=False)

# Solve the weighted bipartite matching
matching = solve_weighted_bipartite(weighted_bipartite_graph)

# Visualization
pos = nx.spring_layout(weighted_bipartite_graph)
nx.draw(weighted_bipartite_graph, pos, with_labels=True, font_weight='bold', node_size=700, node_color='lightblue', font_size=8, font_color='black', edge_color='gray')

# Highlight edges in the matching
matching_edges = [(u, v) for u, v in matching]
nx.draw_networkx_edges(weighted_bipartite_graph, pos, edgelist=matching_edges, edge_color='red', width=2)

plt.show()

In [ ]:
# Quantum functions for QAOA

def custom_mixer_unitary(qc, beta, G):
    """
    Custom mixer unitary for QAOA.
    """
    nqubits = len(G.nodes())
    for i, node in enumerate(G.nodes()):
        qc.rx(2 * beta[i % len(beta)], node)

def qaoa_cost_hamiltonian(G, theta):
    """
    Construct the QAOA cost Hamiltonian circuit.
    """
    nqubits = len(G.nodes())
    nlayers = len(theta) // 2
    beta = theta[:nlayers]
    gamma = theta[nlayers:]

    qc = QuantumCircuit(nqubits)

    # Initial state
    qc.h(range(nqubits))

    for layer in range(nlayers):
        # Problem unitary
        for edge in G.edges():
            qc.rzz(2 * gamma[layer], edge[0], edge[1])
        for node in G.nodes():
            qc.rz(2 * gamma[layer], node)
        qc.barrier()

        # Custom mixer unitary
        custom_mixer_unitary(qc, beta, G)

    return qc

def qaoa_expectation(theta, G, shots=4096):
    """
    Compute the expectation value for QAOA.
    """
    qc = qaoa_cost_hamiltonian(G, theta)
    backend = Aer.get_backend('aer_simulator_statevector')
    backend.shots = shots
    result = execute(qc, backend).result()
    counts = result.get_counts()
    avg = 0
    sum_count = 0
    for bitstring, count in counts.items():
        obj = len(G.edges) - sum(int(bit) for bit in bitstring)
        avg += obj * count
        sum_count += count
    return avg / sum_count

# Optimization of QAOA parameters with bounds
theta_init = np.random.rand(6)  # Initial random parameters (more layers)
bounds = [(0, np.pi)] * len(theta_init)  # Bounds for parameters (between 0 and pi)
res_qaoa = minimize(qaoa_expectation, theta_init, args=(weighted_bipartite_graph,), method='L-BFGS-B', bounds=bounds)

# Visualizing the QAOA results with a deeper circuit
qc_qaoa = qaoa_cost_hamiltonian(weighted_bipartite_graph, res_qaoa.x)
counts_qaoa = execute(qc_qaoa, Aer.get_backend('aer_simulator_statevector')).result().get_counts()
plot_histogram(counts_qaoa, figsize=(15, 15), title="Deeper QAOA Solution")

# Highlighting the optimal matching
matching_qaoa = [(int(u), int(v)) for u, v in counts_qaoa[0].keys() if counts_qaoa[0][(u, v)] > 0]
pos_qaoa = nx.spring_layout(weighted_bipartite_graph)
nx.draw(weighted_bipartite_graph, pos_qaoa, with_labels=True, font_weight='bold', node_size=700, node_color='lightblue', font_size=8, font_color='black', edge_color='gray')
nx.draw_networkx_edges(weighted_bipartite_graph, pos_qaoa, edgelist=matching_qaoa, edge_color='red', width=2)

plt.show()